# Лабораторна робота 3 з "Асиметричних криптосистем та протоколів"
## Тема: Криптосистема Рабіна; Атака на протокол доведення знання без розголошення

**Виконали**\
Дигас Богдан, ФІ-03\
Починок Юрій, ФІ-03

## Підключаємо бібліотеки

In [1720]:
import random
rand = random.SystemRandom()
from sympy.ntheory import jacobi_symbol, legendre_symbol
import math

## Вибираємо бітову довжину $p$ і $q$ - дільників нашого публічного ключа

In [1721]:
bit_length = 256

## Допоміжні функції

### Перевірка на простоту, перетворення числа у бінарному представленні на десяткове та розширений алгоритм Евкліда

In [1722]:

def decomposing_number(n, a):
    exp = n - 1
    while not exp & 1:  # while exp is even
        exp >>= 1  # divide by 2
    if pow(a, exp, n) == 1:
        return True  # number is composite
    while exp < n - 1:
        if pow(a, exp, n) == n - 1:
            return True  # number is composite
        exp <<= 1  # multiply by 2
    return False  # number is probably prime


def miller_rabbin_test(n, k=20):
    for i in range(k):
        a = rand.randrange(1, n - 1)
        if not decomposing_number(n, a):
            return False  # number is composite
    return True  # number is probably prime

def bin_to_dec(bin_n):
    dec_n = 0
    res = 0
    for i in range(len(bin_n)):
        res = bin_n[len(bin_n) - i - 1] * 2 ** i
        dec_n += res
    return dec_n

def extended_gcd(a, b): 
    if a == 0 : 
        return b,0,1       
    gcd,x1,y1 = extended_gcd(b%a, a) 
     
    x = y1 - (b//a) * x1 
    y = x1 
     
    return gcd,x,y 

### Генерація випадкових чисел

In [1723]:
def generate_bit_seq(n):
    seq = [0]*n
    for i in range(n):
        seq[i] = rand.randint(0, 1)
    return seq


def L20(n):
    seq = generate_bit_seq(20)
    result = [0]*n
    for i in range(20):
        result[i] = seq[i]
    for i in range(20,n):
        result[i] = result[i-3]^result[i-5]^result[i-9]^result[i-20]
    return result

### Генерація простого числа

In [1724]:
def generate_prime_number(x):
    res = [1,0,0]
    while(miller_rabbin_test(bin_to_dec(res)) == False):
        res = L20(x)
    return res

### Генерація простих чисел Блюма

In [1725]:
def generate_blum_primes(n):
    p = bin_to_dec(generate_prime_number(n))
    while((p-3) % 4 != 0):
        p = bin_to_dec(generate_prime_number(n))
    
    q = bin_to_dec(generate_prime_number(n))
    while((q-3) % 4 != 0):
        q = bin_to_dec(generate_prime_number(n))
        
    return p, q

### Швидке обчислення квадратного кореня за Блюма $(p, q = 4k + 3, k \in \mathrm{Z})$

In [1726]:
def fast_square_blum(y, p, q): # x^2 = y(mod n), n = p*q, p & q = 4k + 3
    n = p*q
    s_1 = pow(y, (p+1)//4, p) # '//' is for it to be int, not float
    # print("(p+1)//4) = ", (p+1)%4)                                                         # TO DO: MAKE ROOTS WORK
    s_2 = pow(y, (q+1)//4, q)
    # print("(q+1)//4) = ", (q+1)%4)
    _, u, v = extended_gcd(p, q)
    # print("u =", u, "v =", v)
    return (u*p*s_2 + v*q*s_1) % n, (u*p*s_2 - v*q*s_1) % n, ((-1)*u*p*s_2 + v*q*s_1) % n, ((-1)*u*p*s_2 - v*q*s_1) % n         # ++, +-, -+, --

In [1727]:
# Перевірка на уважність)))))))))))

# x = 4
# test_roots = fast_square_blum(x, 19, 11)
# print(test_roots)
# for root in test_roots:
#     print(pow(root, 2, 11*19))

### Форматування та видалення форматування повідомлення

In [1728]:
def format(m,n):
    l = math.ceil((len(bin(n))-2)/8)
    if(math.ceil((len(bin(m))-2)/8)<(l-10)):
        # print(l, math.ceil((len(bin(m))-2)/8))
        r = 2**64-1                             # TO DO: make r random
        x = 255*2**(8*(l-2)) + m*2**64 + r
        # print("Passed the condition")
        return x
    else:
        return m
    
def unformat(m,n):
    l = len(bin(n))-2
    #print(l)
    m = bin(m)
    #print(m,len(m))
    x=''
    #print(l)
    for i in range(10,(len(m)-64)):
        x+=(m[i])
    return int(x,2)

In [1729]:
# Перевірка на уважність)))))))))))

#p, q = generate_blum_primes(bit_length)
# n = p*q
# M = 7532235123452342552455574234234556324546554673185648762345534656545234523452346234623452345234523452345234522345234562346236564
# f = format(M, n)
# print(bin(M))
# print(bin(f))
# # print(n)
# uf = unformat(f, n)
# print(uf)


## Прописуємо інтерфейс користувача

In [1730]:
class User:
    
    __p = None      # private key, key pair (p,q)
    __q = None
    __public_n = None
    __k = None

    def __init__(self):
        self.__p, self.__q = generate_blum_primes(bit_length)
        self.__public_n = self.__p*self.__q
    
    def get_public_key(self):
        return self.__public_n
    
    def Rabin_decrypt(self, C):
        y, c_1, c_2 = C
        roots = fast_square_blum(y, self.__p, self.__q)
        for root in roots:
            root_c_1 = root % 2
            root_c_2 = int(jacobi_symbol(root, self.__public_n) == 1)
            # print(root, root_c_1, jacobi_symbol(root, self.__public_n))
            if(root_c_1 == c_1 and root_c_2 == c_2):
                return unformat(root, self.__public_n)      # Returning M
        print("If you got to this point, there are no useful roots")
        
    def Rabin_sign(self, M):
        while True:         # reformat until we satisfy the condition
            x = format(M, self.__public_n)
            print("Formatting while signing")
            if (jacobi_symbol(x, self.__p) == 1 and jacobi_symbol(x, self.__q) == 1):   # condition: (x, p) == (x, q) == 1 
                break
        # At this point the condition should be satisfied
        
        roots = fast_square_blum(x, self.__p, self.__q)
        return M, roots[rand.randrange(0, 3)] # Return the message and the random root as a pair (M[essage], S[ign])
        

## Прописуємо загальний інтерфейс роботи з користувачем

In [1731]:
def Rabin_encrypt(M, n):
    x = format(M, n)
    y = pow(x, 2, n)    # y = x^2 mod n
    c_1 = x % 2
    c_2 = int(jacobi_symbol(x,n) == 1)
    return (y, c_1, c_2)

# Example: Rabin_encrypt(x, User.get_public_key())

def Rabin_verify(M, S, n):
    supposed_M = pow(S, 2, n)
    return M == unformat(supposed_M, n)

In [1732]:
A = User()
M = 512
C = Rabin_encrypt(M, A.get_public_key())
new_M = A.Rabin_decrypt(C)
print(new_M)

512
